In [1]:
!pip install --upgrade pip
# !pip install --upgrade pandas
!pip install tables   
# necessary for pd.read_hdf()

!pip install ipywidgets
!pip install --upgrade jupyter
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import os
import random
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GroupKFold
import anndata as ad

In [4]:
%matplotlib inline
from tqdm.notebook import tqdm
import gc
import pickle

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## data load

In [5]:
lrz_path = '/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/open-problems-multimodal-3rd-solution/'

model_path_for_now = '/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/'

raw_path =  lrz_path + 'input/raw/'  # '../../../input/raw/'

cite_target_path = model_path_for_now + 'input/target/cite/'   # '../../../input/target/cite/'
cite_feature_path = model_path_for_now + 'input/features/cite/'   # '../../../input/features/cite/'
cite_mlp_path = model_path_for_now + 'model/cite/mlp/'   # '../../../model/cite/mlp/'   # '../../../model/cite/mlp/'
cite_cb_path = model_path_for_now + 'model/cite/cb/'   # '../../../model/cite/cb/'

multi_target_path = lrz_path + 'input/target/multi/'   # '../../../input/target/multi/'
multi_feature_path = lrz_path + 'input/features/multi/'   # '../../../input/features/multi/'
multi_mlp_path = lrz_path + 'model/multi/mlp/'   # '../../../model/multi/mlp/'
multi_cb_path = lrz_path + 'model/multi/cb/'   # '../../../model/multi/cb/'

output_path = lrz_path + 'output/'   # '../../../output/'

## Cite

In [6]:
# get model name
#mlp_model_path = os.listdir(cite_mlp_path)

In [7]:
mlp_model_name = [
    'corr_add_con_imp',
    'corr_last_v3', 
    'corr_c_add_w2v_v1_mish_flg',
    'corr_c_add_w2v_v1_flg',
    'corr_c_add_84_v1',
    'corr_c_add_120_v1',
    'corr_w2v_cell_flg',
    'corr_best_cell_120',
    'corr_cluster_cell',
    'corr_w2v_128',
    'corr_imp_w2v_128',
    'corr_snorm',
    'corr_best_128',
    'corr_best_64',
    'corr_cluster_128',
    'corr_cluster_64',
    'corr_svd_128',
    'corr_svd_64',
             ]

In [8]:
model_name_list = []

for i in mlp_model_name:
    for num, j in enumerate(os.listdir(cite_mlp_path)):
        if i in j:
            model_name_list.append(j)

len(model_name_list)
model_name_list

['cite_mlp_corr_add_con_imp_flg_donor_val_50',
 'cite_mlp_corr_last_v3_flg_donor_val_55',
 'cite_mlp_corr_c_add_w2v_v1_mish_flg_donor_val_66',
 'cite_mlp_corr_c_add_w2v_v1_flg_donor_val_66',
 'cite_mlp_corr_c_add_84_v1_flg_donor_val_47',
 'cite_mlp_corr_c_add_120_v1_flg_donor_val_63',
 'cite_mlp_corr_w2v_cell_flg_donor_val_51',
 'cite_mlp_corr_best_cell_120_flg_donor_val_51',
 'cite_mlp_corr_cluster_cell_flg_donor_val_64',
 'cite_mlp_corr_w2v_128_flg_donor_val_42',
 'cite_mlp_corr_imp_w2v_128_flg_donor_val_38',
 'cite_mlp_corr_snorm_flg_donor_val_39',
 'cite_mlp_corr_best_128_flg_donor_val_45',
 'cite_mlp_corr_best_64_flg_donor_val_50',
 'cite_mlp_corr_cluster_128_flg_donor_val_51',
 'cite_mlp_corr_cluster_64_flg_donor_val_57',
 'cite_mlp_corr_svd_128_flg_donor_val_30',
 'cite_mlp_corr_svd_64_flg_donor_val_38']

In [9]:
weight = [1, 0.3, 1, 1, 1, 1, 1, 1, 1, 0.8, 0.8, 0.8, 0.8, 0.5, 0.5, 0.5, 1, 1, 2, 2]
weight_sum = np.array(weight).sum()
weight_sum

# dict:            model name          input-feature-cite file:    weight
model_feat_dict = {model_name_list[0]:['X_test_add_con_imp.pickle', 1],
                   model_name_list[1]:['X_test_last_v3.pickle', 0.3],
                   model_name_list[2]:['X_test_c_add_w2v_v1.pickle', 1],
                   model_name_list[3]:['X_test_c_add_w2v_v1.pickle', 1],
                   model_name_list[4]:['X_test_c_add_84_v1.pickle', 1],
                   model_name_list[5]:['X_test_c_add_v1.pickle', 1],
                   
                   model_name_list[6]:['X_test_feature_w2v_cell.pickle', 1],
                   model_name_list[7]:['X_test_best_cell_128_120.pickle', 1],
                   model_name_list[8]:['X_test_cluster_cell_128.pickle', 1],
                   
                   model_name_list[9]:['X_test_feature_w2v.pickle', 0.8],
                   model_name_list[10]:['X_test_feature_imp_w2v.pickle',0.8],
                   model_name_list[11]:['X_test_feature_snorm.pickle', 0.8],
                   model_name_list[12]:['X_test_best_128.pickle', 0.8],
                   model_name_list[13]:['X_test_best_64.pickle', 0.5],
                   model_name_list[14]:['X_test_cluster_128.pickle', 0.5],
                   model_name_list[15]:['X_test_cluster_64.pickle', 0.5],
                   model_name_list[16]:['X_test_svd_128.pickle', 1],
                   model_name_list[17]:['X_test_svd_64.pickle', 1],
                   
                   'best_128':['X_test_best_128.pickle', 2],
                   'best_64':['X_test_best_64.pickle', 2],
                  }

In [10]:
# new
for i in model_name_list:
    #i = 'cite_mlp_corr_snorm_flg_donor_val_39'
    try:
        test_file = model_feat_dict[i][0]
        X_test = pd.read_pickle(cite_feature_path  + test_file)   
        print(cite_feature_path  + test_file)
        print(X_test.shape)
    except:
        print('UnpicklingError: ', i)

/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_add_con_imp.pickle
(48203, 925)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_last_v3.pickle
(48203, 843)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_c_add_w2v_v1.pickle
(48203, 843)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_c_add_w2v_v1.pickle
(48203, 843)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_c_add_84_v1.pickle
(48203, 667)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_c_add_v1.pickle
(48203, 811)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_feature_w2v_cell.pickle
(48203, 752)
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/inp

### cite model

In [11]:
def std(x):
    x = np.array(x)
    return (x - x.mean(1).reshape(-1, 1)) / x.std(1).reshape(-1, 1)

In [12]:
class CiteDataset(Dataset):
    
    def __init__(self, feature, target):
        
        self.feature = feature
        self.target = target
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index],
            "y" : self.target[index],
        }
        return d

In [13]:
class CiteDataset_test(Dataset):
    
    def __init__(self, feature):
        self.feature = feature
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index]
        }
        return d

In [14]:
def partial_correlation_score_torch_faster(y_true, y_pred):
    """Compute the correlation between each rows of the y_true and y_pred tensors.
    Compatible with backpropagation.
    """
    y_true_centered = y_true - torch.mean(y_true, dim=1)[:,None]
    y_pred_centered = y_pred - torch.mean(y_pred, dim=1)[:,None]
    cov_tp = torch.sum(y_true_centered*y_pred_centered, dim=1)/(y_true.shape[1]-1)
    var_t = torch.sum(y_true_centered**2, dim=1)/(y_true.shape[1]-1)
    var_p = torch.sum(y_pred_centered**2, dim=1)/(y_true.shape[1]-1)
    return cov_tp/torch.sqrt(var_t*var_p)

def correl_loss(pred, tgt):
    """Loss for directly optimizing the correlation.
    """
    return -torch.mean(partial_correlation_score_torch_faster(tgt, pred))

In [15]:
class CiteModel(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.ReLU(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.ReLU(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.ReLU(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
        
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        return out

In [16]:
class CiteModel_mish(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel_mish, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.Mish(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.Mish(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.Mish(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
    
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        return out

In [17]:
def train_loop(model, optimizer, loader, epoch):
    
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    #loss_fn = nn.MSELoss()
    
    with tqdm(total=len(loader),unit="batch") as pbar:
        pbar.set_description(f"Epoch{epoch}")
        
        for d in loader:
            X = d['X'].to(device)
            y = d['y'].to(device)
            
            logits = model(X)
            loss = correl_loss(logits, y)
            #loss = torch.sqrt(loss_fn(logits, y))
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            pbar.set_postfix({"loss":loss.item()})
            pbar.update(1)

    return model

In [18]:
def valid_loop(model, loader, y_val):
    
    model.eval()
    partial_correlation_scores = []
    oof_pred = []
    
    for d in loader:
        with torch.no_grad():
            val_X = d['X'].to(device).float()
            val_y = d['y'].to(device)
            logits = model(val_X)
            oof_pred.append(logits)
    
    #print(torch.cat(oof_pred).shape, torch.cat(oof_pred).detach().cpu().numpy().shape)
    cor = partial_correlation_score_torch_faster(torch.tensor(y_val).to(device), torch.cat(oof_pred))
    cor = cor.mean().item()
    logits = torch.cat(oof_pred).detach().cpu().numpy()
    
    return logits, cor

In [19]:
def test_loop(model, loader):
    
    model.eval()
    predicts=[]

    for d in tqdm(loader):
        with torch.no_grad():
            X = d['X'].to(device)
            logits = model(X)
            predicts.append(logits.detach().cpu().numpy())
            
    return np.concatenate(predicts)

### pred

In [20]:
pred = np.zeros([48203, 140])

for num, i in enumerate(model_feat_dict.keys()):
    
    print(i)
    
    if 'mlp' in i:

        try:
            test_file = model_feat_dict[i][0]
            test_weight = model_feat_dict[i][1]
            X_test = pd.read_pickle(cite_feature_path  + test_file)   
            print(cite_feature_path  + test_file)
            X_test = np.array(X_test)
            feature_dims = X_test.shape[1]

            test_ds = CiteDataset_test(X_test)
            test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                                         shuffle=False, drop_last=False, num_workers=4)

            if 'mish' in i:
                model = CiteModel_mish(feature_dims)
            else:
                model = CiteModel(feature_dims)

            model = model.to(device)
            model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

            result = test_loop(model, test_dataloader).astype(np.float32)
            result = std(result) * test_weight / weight_sum
            pred += result

            torch.cuda.empty_cache()
        except:
            print('UnpicklingError: ', i)        # TODOOOOOOOOOOOOOO
        
    else:
        test_file = model_feat_dict[i][0]
        test_weight = model_feat_dict[i][1]
        X_test = pd.read_pickle(cite_feature_path  + test_file)
        
        cb_pred = np.zeros([48203, 140])
        
        for t in tqdm(range(140)): 
            cb_model_path = [j for j in os.listdir(cite_cb_path) if f'cb_{t}_{i}' in j][0]
            cb = pickle.load(open(cite_cb_path + cb_model_path, 'rb'))
            cb_pred[:,t] = cb.predict(X_test)
            
        cb_pred = cb_pred.astype(np.float32)
        pred += std(cb_pred) * test_weight / weight_sum
        
        #del cb_pred

cite_mlp_corr_add_con_imp_flg_donor_val_50
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_add_con_imp.pickle


  0%|          | 0/377 [00:00<?, ?it/s]

cite_mlp_corr_last_v3_flg_donor_val_55
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_last_v3.pickle


  0%|          | 0/377 [00:00<?, ?it/s]

cite_mlp_corr_c_add_w2v_v1_mish_flg_donor_val_66
/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/input/features/cite/X_test_c_add_w2v_v1.pickle


...

best_128


  0%|          | 0/140 [00:00<?, ?it/s]

TBB Warning: The number of workers is currently limited to 9. The request for 79 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



best_64


  0%|          | 0/140 [00:00<?, ?it/s]

In [21]:
cite_sub = pd.DataFrame(pred.round(6))
cite_sub

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,-0.325767,-0.487738,-0.351139,1.035394,0.867302,2.263471,3.314403,-0.490992,-0.422089,-0.312560,-0.516754,-0.479289,-0.392685,-0.437807,2.248444,-0.356812,1.858463,1.438221,0.311931,-0.459447,-0.390223,0.357044,-0.487529,-0.386469,2.933681,-0.493526,-0.496832,-0.490686,-0.519472,-0.486443,-0.499483,-0.456832,-0.462952,-0.465901,-0.445472,-0.486322,-0.406220,3.706107,-0.426987,-0.426969,-0.478369,-0.473534,-0.493516,-0.463637,-0.521282,-0.477247,-0.308790,-0.352572,2.829829,-0.491238,-0.471341,-0.467674,-0.427709,-0.501900,-0.094166,0.147856,-0.470777,1.119057,-0.465110,-0.468767,-0.465743,-0.437902,-0.412044,-0.464638,-0.483274,-0.425702,-0.353300,-0.406926,0.377374,-0.491446,-0.499591,-0.521969,-0.460512,3.233928,-0.502028,1.961053,-0.459225,0.279651,-0.455238,-0.469370,0.169536,-0.508457,-0.244526,-0.434981,-0.501910,-0.498316,-0.441767,-0.476339,-0.357719,-0.479157,-0.351639,-0.498732,-0.471366,-0.378029,0.257164,-0.401403,-0.442544,1.844755,-0.465230,1.572834,0.957418,-0.488895,1.072358,-0.422123,0.028856,-0.414253,1.396735,-0.268038,2.183714,-0.345104,-0.337413,0.395187,-0.390905,-0.465734,-0.479009,0.327426,-0.144475,-0.422026,-0.452289,2.622928,-0.450933,0.406673,-0.475251,-0.482181,-0.423303,-0.416871,-0.487996,-0.307121,-0.448958,-0.488162,-0.514007,1.841981,-0.389786,-0.518213,-0.500604,-0.341130,-0.016153,-0.317298,0.038913,0.358591
1,-0.293307,-0.470185,-0.351417,1.042340,0.918223,2.295353,3.557569,-0.485068,-0.415240,-0.313825,-0.507285,-0.467309,-0.390135,-0.437772,2.081139,-0.379274,1.817812,1.527190,0.218958,-0.443928,-0.396843,0.337870,-0.473521,-0.378367,2.997932,-0.475288,-0.487909,-0.482030,-0.519096,-0.478371,-0.486569,-0.440771,-0.453228,-0.449638,-0.432872,-0.484526,-0.406047,3.649258,-0.414698,-0.415922,-0.476712,-0.462437,-0.486182,-0.507216,-0.513428,-0.466063,-0.308042,-0.362187,2.793647,-0.483065,-0.458920,-0.464745,-0.472144,-0.494432,-0.116090,0.168782,-0.459241,0.760353,-0.473271,-0.456876,-0.456543,-0.430272,-0.406301,-0.449704,-0.480652,-0.419016,-0.357765,-0.404320,0.361015,-0.486164,-0.483510,-0.508212,-0.455934,3.269518,-0.497218,1.836228,-0.448956,0.262899,-0.452102,-0.455511,0.015679,-0.488521,-0.228284,-0.424335,-0.491802,-0.481530,-0.433237,-0.465633,-0.349264,-0.457304,-0.351280,-0.485360,-0.462076,-0.375112,0.195090,-0.387814,-0.433895,1.852686,-0.458732,1.629971,0.989916,-0.481465,1.042104,-0.415026,-0.026993,-0.408080,1.243898,-0.315948,2.307294,-0.338717,-0.338424,0.279150,-0.370051,-0.454253,-0.466977,0.202053,-0.207233,-0.402973,-0.442968,2.643998,-0.447381,0.725545,-0.461757,-0.474494,-0.407510,-0.405384,-0.473229,-0.275020,-0.438769,-0.481086,-0.503055,1.824850,-0.384038,-0.505780,-0.490800,-0.358872,-0.072174,-0.322904,0.059740,0.321151
2,-0.337575,-0.449081,-0.328770,1.541549,1.271137,1.486816,4.438024,-0.463581,-0.391279,-0.423727,-0.490480,-0.432064,-0.413590,-0.419570,3.426018,-0.273052,1.050578,1.232067,-0.147889,-0.459557,-0.391296,0.349071,-0.452589,-0.377862,2.946249,-0.457357,-0.465505,-0.456003,-0.552835,-0.449007,-0.464044,-0.396615,-0.427664,-0.421790,-0.406941,-0.461765,-0.376789,3.081319,-0.390267,-0.414918,-0.445042,-0.445852,-0.436071,-0.241519,-0.477286,-0.428363,-0.259145,-0.387482,0.834384,-0.462517,-0.428094,-0.450838,-0.397854,-0.472625,-0.105831,0.155859,-0.442649,0.735279,-0.463939,-0.440942,-0.214496,-0.391008,-0.372151,-0.421942,-0.428422,-0.395630,-0.315540,-0.405780,0.398103,-0.453094,-0.445499,-0.482222,-0.441426,1.487004,-0.439823,1.227368,-0.422020,0.056478,-0.423029,-0.428986,-0.157021,-0.457496,-0.1489

In [22]:
#cite_sub.to_csv('../../../../../summary/output/submit/cite_submit.csv')

In [23]:
# model #16: cite_mlp_corr_svd_128_flg_donor_val_30
pred_16 = np.zeros([48203, 140])

i = 'cite_mlp_corr_svd_128_flg_donor_val_30'
        
test_file = model_feat_dict[i][0]
test_weight = model_feat_dict[i][1]
X_test = pd.read_pickle(cite_feature_path  + test_file)    
X_test = np.array(X_test)
feature_dims = X_test.shape[1]

test_ds = CiteDataset_test(X_test)
test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                              shuffle=False, drop_last=False, num_workers=4)

if 'mish' in i:
    model = CiteModel_mish(feature_dims)
else:
    model = CiteModel(feature_dims)
    
model = model.to(device)
model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

result = test_loop(model, test_dataloader).astype(np.float32)
# result = std(result) * test_weight / weight_sum  # double check 
pred_16 += result

torch.cuda.empty_cache()
        
pd.DataFrame(pred_16)   # double check train_cite_targets.h5  -> omnipath -> then maybe shap
# TODO SHAP on this

  0%|          | 0/377 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,-1.066433,-1.500550,-1.354774,1.106190,0.869803,3.481040,5.067838,-1.570396,-1.444474,-1.320852,-1.612052,-1.534098,-1.428519,-1.496802,3.095008,-1.384286,2.648872,1.938160,-0.093022,-1.635664,-1.428120,-0.072564,-1.464633,-1.333522,4.528148,-1.577123,-1.570215,-1.564498,-1.633450,-1.551511,-1.606081,-1.488960,-1.542044,-1.542291,-1.515924,-1.584971,-1.483199,6.078821,-1.523234,-1.523154,-1.628240,-1.568818,-1.554284,-1.576392,-1.607019,-1.563617,-1.300677,-1.282699,4.529342,-1.497951,-1.504461,-1.616715,-1.604993,-1.590265,-0.913128,-0.373880,-1.571010,1.285669,-1.561719,-1.562545,-1.531397,-1.546362,-1.493735,-1.614805,-1.581547,-1.436415,-1.352806,-1.433268,0.004768,-1.507400,-1.582382,-1.650515,-1.553528,5.308838,-1.614918,2.937887,-1.565187,-0.132013,-1.512913,-1.525913,-0.179205,-1.617044,-1.141805,-1.497630,-1.589817,-1.619898,-1.517950,-1.527373,-1.324592,-1.536239,-1.349147,-1.607808,-1.596236,-1.449520,-0.069496,-1.395786,-1.459246,2.709843,-1.504532,2.281003,1.127239,-1.582563,1.390550,-1.485366,-0.814275,-1.483867,1.818746,-1.166362,3.551317,-1.324322,-1.333103,-0.007823,-1.370018,-1.791438,-1.608289,-0.100262,-0.915906,-1.476855,-1.569585,4.021511,-1.475899,0.135406,-1.601376,-1.499214,-1.508077,-1.504729,-1.617144,-1.285857,-1.497512,-1.602304,-1.585749,2.483783,-1.404202,-1.656756,-1.628250,-1.330432,-1.008113,-1.255932,-0.622936,-0.130905
1,-1.067110,-1.506955,-1.399288,1.075525,0.997081,3.506621,5.481764,-1.564466,-1.460833,-1.301538,-1.630497,-1.572419,-1.490347,-1.531179,3.168335,-1.427267,2.470253,2.130677,-0.441134,-1.637434,-1.472109,-0.058242,-1.472134,-1.346540,4.738452,-1.580705,-1.617836,-1.605957,-1.660571,-1.576319,-1.645332,-1.515913,-1.583853,-1.545092,-1.542181,-1.602096,-1.507277,6.134613,-1.544583,-1.598276,-1.658123,-1.569817,-1.591216,-1.618024,-1.622394,-1.573258,-1.365325,-1.374641,4.661466,-1.514943,-1.525232,-1.650555,-1.612984,-1.614313,-1.050634,-0.368892,-1.596666,0.777738,-1.589263,-1.581116,-1.578929,-1.556408,-1.516185,-1.611275,-1.630935,-1.470598,-1.411295,-1.485476,-0.118224,-1.505585,-1.588891,-1.651967,-1.588987,5.179552,-1.639723,2.675957,-1.584337,-0.139389,-1.552935,-1.552968,-0.562470,-1.613981,-1.148527,-1.525604,-1.601094,-1.632247,-1.561806,-1.547703,-1.396585,-1.549065,-1.390192,-1.631667,-1.603218,-1.467852,-0.253815,-1.409238,-1.494805,2.745513,-1.549431,2.249057,0.980224,-1.594143,1.342945,-1.517599,-0.857558,-1.512639,1.533982,-1.339978,3.737477,-1.391074,-1.371431,-0.343284,-1.380614,-1.574346,-1.623899,-0.251694,-1.224326,-1.529244,-1.581402,4.151693,-1.492653,0.829439,-1.611794,-1.553943,-1.531965,-1.514009,-1.643932,-1.266751,-1.511899,-1.586327,-1.620505,2.777342,-1.404117,-1.657573,-1.687846,-1.422966,-0.780818,-1.299758,-0.628505,-0.121618
2,-1.010656,-1.271471,-1.207577,2.074188,1.614956,2.007590,6.589466,-1.374983,-1.260541,-1.450437,-1.464440,-1.332188,-1.378543,-1.357404,4.944262,-1.135714,0.924946,1.518677,-0.864070,-1.520523,-1.308277,0.001427,-1.352170,-1.241024,4.492778,-1.413115,-1.387910,-1.474926,-1.551057,-1.376166,-1.415526,-1.233463,-1.391570,-1.319213,-1.332545,-1.431528,-1.287814,4.748064,-1.345603,-1.431644,-1.396456,-1.397756,-1.360939,-1.085002,-1.420398,-1.334369,-1.064852,-1.288315,1.083776,-1.353123,-1.357820,-1.394112,-1.357486,-1.375918,-0.872777,-0.296593,-1.410997,0.365104,-1.434169,-1.399326,-0.980195,-1.282730,-1.286807,-1.391266,-1.348446,-1.279712,-1.213837,-1.296896,0.074692,-1.323143,-1.357035,-1.430854,-1.481354,2.077808,-1.360545,1.681203,-1.313775,-0.382094,-1.374308,-1.361059,-0.

In [24]:
train_cite_targets = pd.read_hdf('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/neurips_competition_data/train_cite_targets.h5')    # train_cite_targets.h5 - Surface protein levels for the same cells that have been dsb normalized.
print(train_cite_targets.shape)   # more rows; how to match rows -> metadata for cell_id
train_cite_targets  

(70988, 140)


gene_id,CD86,CD274,CD270,CD155,CD112,CD47,CD48,CD40,CD154,CD52,CD3,CD8,CD56,CD19,CD33,CD11c,HLA-A-B-C,CD45RA,CD123,CD7,CD105,CD49f,CD194,CD4,CD44,CD14,CD16,CD25,CD45RO,CD279,TIGIT,Mouse-IgG1,Mouse-IgG2a,Mouse-IgG2b,Rat-IgG2b,CD20,CD335,CD31,Podoplanin,CD146,IgM,CD5,CD195,CD32,CD196,CD185,CD103,CD69,CD62L,CD161,CD152,CD223,KLRG1,CD27,CD107a,CD95,CD134,HLA-DR,CD1c,CD11b,CD64,CD141,CD1d,CD314,CD35,CD57,CD272,CD278,CD58,CD39,CX3CR1,CD24,CD21,CD11a,CD79b,CD244,CD169,integrinB7,CD268,CD42b,CD54,CD62P,CD119,TCR,Rat-IgG1,Rat-IgG2a,CD192,CD122,FceRIa,CD41,CD137,CD163,CD83,CD124,CD13,CD2,CD226,CD29,CD303,CD49b,CD81,IgD,CD18,CD28,CD38,CD127,CD45,CD22,CD71,CD26,CD115,CD63,CD304,CD36,CD172a,CD72,CD158,CD93,CD49a,CD49d,CD73,CD9,TCRVa7.2,TCRVd2,LOX-1,CD158b,CD158e1,CD142,CD319,CD352,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
cell_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
45006fe3e4c8,1.167804,0.622530,0.106959,0.324989,3.331674,6.426002,1.480766,-0.728392,-0.468851,-0.073285,0.083974,-0.009263,-0.806745,-0.265419,3.752507,0.148752,5.152736,1.605306,1.289909,-0.032050,-0.011964,0.786215,-0.179121,1.620421,9.063515,-0.426230,0.835079,0.252523,-0.832419,-0.398058,-0.716263,0.132499,-0.034390,0.088057,1.009419,0.124132,0.146069,14.598290,0.068552,-0.311110,0.371351,0.258717,0.238117,0.099418,1.264052,0.353834,0.694085,1.372086,6.545119,-0.411364,-0.812418,1.899534,-0.288912,-0.094387,0.323964,1.418235,-0.094707,3.591847,0.998478,0.823167,-0.213062,-0.826575,0.172696,-1.634767,-0.083557,0.116275,1.112164,0.946800,1.390265,0.849144,0.096078,-0.485744,-0.209445,5.968264,0.136596,5.984627,-0.381260,0.497469,-0.173429,-0.331172,2.518413,1.316934,-0.349472,-0.206551,-0.385590,0.426353,1.095887,-0.240434,0.230145,1.181182,-0.031258,-0.936979,-0.019728,0.698779,1.338748,0.107811,0.581864,3.531209,0.355473,3.451938,1.641979,-0.125480,0.051798,-0.693440,0.783536,0.486164,2.608011,0.006995,8.438908,1.047601,0.721115,2.326464,0.250484,0.666030,1.345943,-0.017324,1.489720,-0.426018,-0.201101,7.087329,-0.031630,1.799062,-0.268062,1.411464,0.087309,0.497463,0.169145,0.982767,0.003275,0.446516,-0.448390,3.220174,-0.533004,0.674956,-0.006187,0.682148,1.398105,0.414292,1.780314,0.548070
d02759a80ba2,0.818970,0.506009,1.078682,6.848758,3.524885,5.279456,4.930438,2.069372,0.333652,-0.468088,0.869646,-0.367452,1.772582,-0.688032,3.032772,7.559227,4.226589,2.831112,5.347887,-0.291308,1.038259,2.040388,-0.491867,-0.039604,17.513119,0.649578,-0.534067,0.448366,0.361228,-0.150670,-0.539296,-0.338439,0.495782,1.596701,-0.520776,-0.291943,0.113660,13.131903,-0.399182,-0.680874,0.490980,0.476323,-0.125623,0.400299,0.457753,0.927307,1.045198,0.412847,7.070779,0.022260,0.458339,-0.566680,0.129248,-0.396126,2.048669,0.875867,0.218050,4.774466,-0.534997,1.377367,-0.527461,0.493524,0.790165,0.151704,-0.450515,-0.310673,-0.828291,-0.206173,1.732723,-0.158920,0.441403,-0.291708,1.495075,11.771707,-0.555983,7.495656,0.646417,4.111932,-0.525074,0.174254,3.876090,0.217874,-0.251619,0.718693,-0.027717,-0.014002,1.250384,0.210187,0.862488,-0.470985,0.172785,-0.243277,0.421166,1.476295,2.382426,-0.189925,0.119327,6.511853,0.796505,4.606451,4.328416,1.383827,6.605588,-0.535279,1.277863,0.817675,2.821027,-0.356294,12.072031,-0.545996,0.195914,4.415056,0.633260,-0.259919,0.170528,-0.461403,-0.193964,0.668466,0.689649,6.408119,0.519766,6.157828,-0.640918,1.112936,0.521161,0.045560,0.836664,0.524862,1.570606,0.021655,0.323613,8.407108,0.131301,0.047607,-0.243628,0.547864,1.832587,0.982308,2.736507,2.184063
c016c6b0efa5,-0.356703,-0.422261,-0.824493,1.137495,0.518924,7.221962,-0.375034,1.738071,0.142919,-0.971460,-0.834109,-0.267854,0.223893,-0.570519,3.232903,-0.117146,2.874473,-0.247663,0.227811,-0.219218,0.117868,1.505893,0.958283,0.720971,6.064536,-0.704153,0.589955,-0.040713,-0.877728,0.129421,-0.143044,0.396504,-0.369433,0.307419,-0.400956,0.184605,0.636807,4.339216,0.05306

In [25]:
metadata = pd.read_csv('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/neurips_competition_data/metadata.csv')
metadata[metadata['cell_id'].isin(['45006fe3e4c8','d02759a80ba2','c016c6b0efa5','ba7f733a4f75','fbcf2443ffb2'])]

,cell_id,day,donor,cell_type,technology
28043,45006fe3e4c8,2,32606,HSC,citeseq
28044,d02759a80ba2,2,32606,HSC,citeseq
28045,c016c6b0efa5,2,32606,EryP,citeseq
28046,ba7f733a4f75,2,32606,NeuP,citeseq
28047,fbcf2443ffb2,2,32606,EryP,citeseq


In [26]:
# filter dataset by IDs in list
metadata_filtered = metadata[metadata['cell_id'].isin(train_cite_targets.index.values)].set_index('cell_id')
metadata_filtered.head(3)

,day,donor,cell_type,technology
cell_id,,,,
45006fe3e4c8,2,32606,HSC,citeseq
d02759a80ba2,2,32606,HSC,citeseq
c016c6b0efa5,2,32606,EryP,citeseq


In [27]:
train_cite_targets = ad.AnnData(train_cite_targets, obs=metadata_filtered)
train_cite_targets

AnnData object with n_obs × n_vars = 70988 × 140
    obs: 'day', 'donor', 'cell_type', 'technology'

In [43]:
# run after preprocessing below
# cite_sub['cell_id']
pd.DataFrame(cite_sub)[0]

0        c2150f55becb
1        65b7edf8a4da
2        c1b26cb1057b
3        917168fa6f83
4        2b29feeca86d
             ...     
48658    a9b4d99f1f50
48659    0e2c1d0782af
48660    a3cbc5aa0ec3
48661    75b350243add
48662    ad5a949989b2
Name: 0, Length: 48663, dtype: object

In [44]:
target_cell_ids = train_cite_targets.obs.index.tolist()
target_cell_ids[:10]  # 70988

print(set(pd.DataFrame(cite_sub)[0]).issubset(set(target_cell_ids)))


mask = pd.DataFrame(cite_sub)[0].isin(target_cell_ids)
result = pd.DataFrame(cite_sub).loc[mask]

result

False


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140


In [45]:
set(test_sub_ids).issubset(set(target_cell_ids))

False

## Multi

In [ ]:
mlp_model_name = [
    'multi_mlp_all_con_16',
    'multi_mlp_all_con_32', 
    'multi_mlp_all_binary_16',
    'multi_mlp_all_last_cluster',
    'multi_mlp_all_lsi_w2v_col_128_flg',
    'multi_mlp_all_lsi_w2v_128_flg',
    'multi_mlp_all_lsi_128_flg',
    'multi_mlp_all_lsi_w2v_col_64_flg',
    'multi_mlp_all_lsi_w2v_64_flg',
    'multi_mlp_all_lsi_64_flg',
    'multi_mlp_all_okapi_128_flg',
    'multi_mlp_all_okapi_64_flg',
    'multi_mlp_all_colmean_64_flg',
    'multi_mlp_corr_con_16_flg',
    'multi_mlp_corr_con_32_flg',
    'multi_mlp_corr_binary_16',
    'multi_mlp_corr_lsi_add_lc_svd_flg',
    
    'multi_mlp_corr_lsi_w2v_col_128_flg',
    'multi_mlp_corr_lsi_w2v_col_64_flg',
    'multi_mlp_corr_lsi_w2v_128_flg',
    'multi_mlp_corr_lsi_w2v_64_flg',
    
    'multi_mlp_corr_lsi_128_flg',
    'multi_mlp_corr_lsi_64_flg',
    
    'multi_mlp_corr_colmean_64_flg',
    'multi_mlp_corr_okapi_w2v_64_flg',
    'multi_mlp_corr_okapi_64_flg',
    
             ]

In [ ]:
model_name_list = []

for i in mlp_model_name:
    for num, j in enumerate(os.listdir(multi_mlp_path)):
        if i in j:
            model_name_list.append(j)

print(len(model_name_list))
model_name_list

In [ ]:
weight = [2.5, 2.5, 2.5, 1.2, 1.2, 1.2, 1, 
          1.5, 1.5, 2.5, 0.5, 0.5, 0.5, 
          2.5, 2.5, 1.8, 0.8, 1, 0.8, 1 ,0.8, 1, 0.3, 
          0.3, 0.3, 0.3, 0.2, 0.2, 0.2]
weight_sum = np.array(weight).sum()
weight_sum

model_feat_dict = {model_name_list[0]:['multi_test_con_16.pickle', 2.5],
                   model_name_list[1]:['multi_test_con_32.pickle', 2.5],
                   model_name_list[2]:['multi_test_binary_16.pickle', 2.5],
                   
                   model_name_list[3]:['multi_test_okapi_64_last_cluster.pickle', 1.2],
                   model_name_list[4]:['multi_test_lsi_w2v_col_128.pickle', 1.2],
                   model_name_list[5]:['multi_test_lsi_w2v_128.pickle', 1.2],
                   model_name_list[6]:['multi_test_okapi_lsi_128.pickle', 1],
                   
                   model_name_list[7]:['multi_test_lsi_w2v_col_64.pickle', 1.5],
                   model_name_list[8]:['multi_test_lsi_w2v_64.pickle', 1.5],
                   model_name_list[9]:['multi_test_okapi_lsi_64.pickle', 2.5],
                   
                   model_name_list[10]:['multi_test_okapi_feature_128.pickle', 0.5],
                   model_name_list[11]:['multi_test_okapi_feature_64.pickle', 0.5],
                   model_name_list[12]:['multi_test_okapi_w2v_col_64.pickle', 0.5],
                   
                   model_name_list[13]:['multi_test_con_16.pickle', 2.5],
                   model_name_list[14]:['multi_test_con_32.pickle', 2.5],
                   model_name_list[15]:['multi_test_binary_16.pickle', 1.8],
                   model_name_list[16]:['multi_test_lc_addsvd_64.pickle', 0.8],
                   
                   model_name_list[17]:['multi_test_lsi_w2v_col_128.pickle', 1],
                   model_name_list[18]:['multi_test_lsi_w2v_col_64.pickle', 0.8],
                   model_name_list[19]:['multi_test_lsi_w2v_128.pickle', 1],
                   model_name_list[20]:['multi_test_lsi_w2v_64.pickle', 0.8],
                   model_name_list[21]:['multi_test_okapi_lsi_128.pickle', 1],
                   model_name_list[22]:['multi_test_okapi_lsi_64.pickle', 0.3],
                   
                   model_name_list[23]:['multi_test_okapi_w2v_col_64.pickle', 0.3],
                   model_name_list[24]:['multi_test_okapi_w2v_64.pickle', 0.3],
                   model_name_list[25]:['multi_test_okapi_feature_64.pickle', 0.3],
                   
                   'lsi_128':['multi_test_okapi_lsi_128.pickle', 0.2],
                   'lsi_64':['multi_test_okapi_lsi_64.pickle', 0.2],
                   'lsi_w2v_col_64':['multi_test_lsi_w2v_col_64.pickle', 0.2],
                  }


### multi model

In [ ]:
class MultiDataset(Dataset):
    
    def __init__(self, feature, target):
        
        self.feature = feature
        self.target = target
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index],
            "y" : self.target[index],
        }
        return d

In [ ]:
class MultiDataset_test(Dataset):
    
    def __init__(self, feature):
        self.feature = feature
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index]
        }
        return d

In [ ]:
def partial_correlation_score_torch_faster(y_true, y_pred):
    """Compute the correlation between each rows of the y_true and y_pred tensors.
    Compatible with backpropagation.
    """
    y_true_centered = y_true - torch.mean(y_true, dim=1)[:,None]
    y_pred_centered = y_pred - torch.mean(y_pred, dim=1)[:,None]
    cov_tp = torch.sum(y_true_centered*y_pred_centered, dim=1)/(y_true.shape[1]-1)
    var_t = torch.sum(y_true_centered**2, dim=1)/(y_true.shape[1]-1)
    var_p = torch.sum(y_pred_centered**2, dim=1)/(y_true.shape[1]-1)
    return cov_tp/torch.sqrt(var_t*var_p)

def correl_loss(pred, tgt):
    """Loss for directly optimizing the correlation.
    """
    return -torch.mean(partial_correlation_score_torch_faster(tgt, pred))


def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

In [ ]:
class MultiModel(nn.Module):
    
    def __init__(self, feature_num):
        super(MultiModel, self).__init__()
        
        self.layer_seq_128 = nn.Sequential(nn.Linear(feature_num, 128),
                                           nn.LayerNorm(128),
                                           nn.ReLU(),
                                      )
        
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                           nn.LayerNorm(64),
                                           nn.ReLU(),
                                      )
        
        self.layer_seq_32 = nn.Sequential(nn.Linear(64, 32),
                                   nn.LayerNorm(32),
                                   nn.ReLU(),
                              )
        
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 8),
                                         nn.LayerNorm(8),
                                         nn.ReLU(),
                                      )
        
        self.head = nn.Linear(128 + 64 + 32 + 8, target_num)
                   
    def forward(self, X, y=None):
        
        X_128 = self.layer_seq_128(X)
        X_64 = self.layer_seq_64(X_128)
        X_32 = self.layer_seq_32(X_64)
        X_8 = self.layer_seq_8(X_32)
        X = torch.cat([X_128, X_64, X_32, X_8], axis = 1)
        out = self.head(X)
        
        return out

In [ ]:
def train_loop(model, optimizer, loader, epoch):
    
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    loss_fn = nn.MSELoss()
    
    with tqdm(total=len(loader),unit="batch") as pbar:
        pbar.set_description(f"Epoch{epoch}")
        
        for d in loader:
            X = d['X'].to(device).float()
            y = d['y'].to(device)
            
            logits = model(X)
            #loss = correl_loss(logits, y)
            loss = torch.sqrt(loss_fn(logits, y))
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            pbar.set_postfix({"loss":loss.item()})
            pbar.update(1)

    return model

In [ ]:
def valid_loop(model, loader, y_val):
    
    model.eval()
    partial_correlation_scores = []
    oof_pred = []
    loss_fn = nn.MSELoss()
    
    for d in loader:
        with torch.no_grad():
            val_X = d['X'].to(device).float()
            val_y = d['y'].to(device)
            logits = model(val_X)
            #oof_pred.append(logits.detach().cpu().numpy())
            oof_pred.append(logits)
    
    y_val = torch.tensor(y_val).to(device)
    logits = torch.cat(oof_pred)
    #print(logits.shape, y_val.shape)
    loss = torch.sqrt(loss_fn(logits, y_val))
    logits = logits.detach().cpu().numpy()
    
    return logits, loss

In [ ]:
def test_loop(model, loader):
    
    model.eval()
    predicts=[]

    for d in tqdm(loader):
        with torch.no_grad():
            X = d['X'].to(device).float()
            logits = model(X)
            predicts.append(logits.detach().cpu().numpy())
            
    return np.concatenate(predicts)

In [ ]:
pred = np.zeros([55935, 23418])
svd = pickle.load(open(multi_target_path + 'multi_all_target_128.pkl', 'rb'))

for num, i in enumerate(model_feat_dict.keys()):
    
    print(i)
    
    if 'mlp' in i:
        
        test_file = model_feat_dict[i][0]
        test_weight = model_feat_dict[i][1]
        X_test = pd.read_pickle(multi_feature_path  + test_file)    
        X_test = np.array(X_test)
        feature_dims = X_test.shape[1]

        test_ds = MultiDataset_test(X_test)
        test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                                     shuffle=False, drop_last=False, num_workers=4)
        
        if 'all' in i:
            target_num = 23418
        else:
            target_num = 128
        
        model = MultiModel(feature_dims)    
        model = model.to(device)
        model.load_state_dict(torch.load(f'{multi_mlp_path}/{i}'))
        
        result = test_loop(model, test_dataloader).astype(np.float32)
        
        if 'all' not in i:
            result = result@svd.components_
                
        result = result * test_weight / weight_sum
        pred += result

        torch.cuda.empty_cache()
        
    else:
        test_file = model_feat_dict[i][0]
        test_weight = model_feat_dict[i][1]
        X_test = pd.read_pickle(multi_feature_path  + test_file)
        
        cb_pred = np.zeros([55935, 128])
        
        for t in tqdm(range(128)): 
            cb_model_path = [j for j in os.listdir(multi_cb_path) if f'cb_{t}_{i}' in j][0]
            cb = pickle.load(open(multi_cb_path + cb_model_path, 'rb'))
            cb_pred[:,t] = cb.predict(X_test)
            
        cb_pred = cb_pred.astype(np.float32)
        cb_pred = cb_pred@svd.components_
        pred += cb_pred * test_weight / weight_sum
        
        #del cb_pred

In [ ]:
multi_sub = pd.DataFrame(pred.round(6)).astype(np.float32)

In [ ]:
del pred
gc.collect()

## Postprocess

In [31]:
preprocess_path = lrz_path + 'input/preprocess/'

#### first: fix cite output

In [32]:
test_sub_ids = np.load(preprocess_path + "test_cite_inputs_idxcol.npz", allow_pickle=True)
test_sub_ids = test_sub_ids["index"]
test_raw_ids = np.load(preprocess_path + "test_cite_raw_inputs_idxcol.npz", allow_pickle=True)
test_raw_ids = test_raw_ids["index"]
test_raw_ids

array(['83d6659a6a32', 'd98594f13d2e', '5f93d8ffc72f', ...,
       'a3cbc5aa0ec3', '75b350243add', 'ad5a949989b2'], dtype=object)

In [33]:
len(test_raw_ids)
cite_sub.shape

(48203, 140)

In [34]:
test_cite_df = pd.DataFrame(test_sub_ids, columns = ['cell_id'])
cite_sub['cell_id'] = test_raw_ids# .tolist()
test_cite_df = test_cite_df.merge(cite_sub, on = 'cell_id', how = 'left')
test_cite_df.fillna(0, inplace = True)
# test_cite_df.drop(['cell_id'], axis = 1, inplace = True)

cite_sub = test_cite_df.copy()


In [35]:
cite_sub

,cell_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,c2150f55becb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,65b7edf8a4da,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,c1b26cb1057b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

### preprocess

In [36]:
sub = pd.read_csv('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/neurips_competition_data/sample_submission.csv')  
eval_ids = pd.read_csv('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/neurips_competition_data/evaluation_ids.csv') 

cite_cols = pd.read_csv(preprocess_path + "cite_test_cols.csv") 
cite_index = pd.read_csv(preprocess_path + "cite_test_indexs.csv") 
cite_index.columns = ['cell_id']

# multi_cols = pd.read_csv(preprocess_path + "multi/multi_test_cols.csv") 
# multi_index = pd.read_csv(preprocess_path + "multi/multi_test_indexs.csv") 
# multi_index.columns = ['cell_id']

submission = pd.Series(name='target',index=pd.MultiIndex.from_frame(eval_ids), dtype=np.float32)


In [37]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

### multi

In [ ]:
multi_sub = np.array(multi_sub)

In [ ]:
cell_dict = dict((k,v) for v,k in enumerate(np.array(multi_index['cell_id'])))
assert len(cell_dict)  == len(multi_index['cell_id'])

gene_dict = dict((k,v) for v,k in enumerate(np.array(multi_cols['gene_id']))) 
assert len(gene_dict)  == len(multi_cols['gene_id'])

eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)
submission.iloc[valid_multi_rows] = multi_sub[eval_ids_cell_num[valid_multi_rows].to_numpy(),
                                                 eval_ids_gene_num[valid_multi_rows].to_numpy()]

### cite

In [38]:
cite_sub = np.array(cite_sub)
cite_sub

array([['c2150f55becb', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['65b7edf8a4da', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['c1b26cb1057b', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       ['a3cbc5aa0ec3', -0.40358, -0.082392, ..., -0.334004, -0.23979,
        0.332295],
       ['75b350243add', -0.611163, -0.492633, ..., -0.519184, 2.048543,
        0.840033],
       ['ad5a949989b2', -0.582459, -0.439802, ..., -0.516436, 2.170141,
        0.980844]], dtype=object)

In [39]:
cell_dict = dict((k,v) for v,k in enumerate(np.array(cite_index['cell_id'])))
assert len(cell_dict)  == len(cite_index['cell_id'])

gene_dict = dict((k,v) for v,k in enumerate(np.array(cite_cols['gene_id']))) 
assert len(gene_dict)  == len(cite_cols['gene_id'])

eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [40]:
submission.iloc[valid_multi_rows] = cite_sub[eval_ids_cell_num[valid_multi_rows].to_numpy(),
                                                 eval_ids_gene_num[valid_multi_rows].to_numpy()]

### make submission

In [41]:
# submission = submission.round(6)
submission = pd.DataFrame(submission, columns = ['target'])
submission = submission.reset_index()

In [42]:
submission[['row_id', 'target']]#.dropna()        #.to_csv(output_path + 'submission.csv', index = False)

,row_id,target
0,0,c2150f55becb
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
65744175,65744175,NaN
65744176,65744176,NaN
65744177,65744177,NaN
65744178,65744178,NaN


In [ ]:
#!kaggle competitions submit -c open-problems-multimodal -f $sub_name_csv -m $message